In [1]:
# THIS IS A BAD EXAMPLE LOOK INTO IT!

'/businessindustryandtrade/changestobusiness/mergersandacquisitions/datasets/mergersandacquisitionsinvolvingukcompanies'

'/businessindustryandtrade/changestobusiness/mergersandacquisitions/datasets/mergersandacquisitionsinvolvingukcompanies'

In [1]:
import torch
import pandas as pd
import numpy as np
import pickle, logging, spacy, sys, os, json, requests
import matplotlib.pyplot as plt

from helpers.classes import Collection
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
import spacy
nlp = spacy.load('en_core_web_trf')

from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer()
vectorizer = TfidfVectorizer(ngram_range=(2,3))

import nltk
stopwords = set(nltk.corpus.stopwords.words('english'))

In [3]:
from helpers.cloze_generation import generate_clozes_from_point, named_entity_answer_generator as ne_answer_generator, noun_phrase_answer_generator as np_answer_generator

from helpers.table_processing import preprocess_table, read_process_table, find_relevant_column_header, find_relevant_content
from helpers.t5_language_model import summarise_t5_results


df = pd.read_pickle('pickles/dataset_20210625_184837.pkl')
clozes_df = pd.read_json('pickles/clozes_20210807_165700.json')

In [4]:
ALL_DATA = os.listdir('datasets/')
ALL_DATA = ['/' + x.replace('_', '/')[:-4] for x in ALL_DATA]

In [5]:
usability = []
for entry in tqdm(ALL_DATA):
    
    try:
        df_iterator = read_process_table(entry, file_ext='.xls')
        
        done = False
        
        df = next(df_iterator, False)
        sheet_name_counter = 0

        if df is None:
            df = False

        while df is not False:
            N, D = df.shape
            usability.append({'rows': N, 'columns': D, 'path': entry, 'sheet_name_counter': sheet_name_counter})
            
            df = next(df_iterator, False)
            sheet_name_counter += 1
            # there are some excel files that have empty sheets...
            # only way I managed to stop it 
            if df is None: 
                df = False
    
    except (ValueError, KeyError, AttributeError, FileNotFoundError):
        usability.append(
            {'rows': -1, 'columns': -1, 'path': entry, 'sheet_name_counter': -1})
        continue


  1%|          | 10/958 [00:00<01:05, 14.36it/s]/Users/pafitis/miniforge3/envs/comp0087/lib/python3.9/site-packages/openpyxl/reader/workbook.py:88: UserWarning: File contains an invalid specification for Information. This will be removed
  warn(msg)
/Users/pafitis/miniforge3/envs/comp0087/lib/python3.9/site-packages/openpyxl/reader/workbook.py:88: UserWarning: File contains an invalid specification for Contents. This will be removed
  warn(msg)
/Users/pafitis/miniforge3/envs/comp0087/lib/python3.9/site-packages/openpyxl/reader/workbook.py:88: UserWarning: File contains an invalid specification for North East. This will be removed
  warn(msg)
/Users/pafitis/miniforge3/envs/comp0087/lib/python3.9/site-packages/openpyxl/reader/workbook.py:88: UserWarning: File contains an invalid specification for North West. This will be removed
  warn(msg)
/Users/pafitis/miniforge3/envs/comp0087/lib/python3.9/site-packages/openpyxl/reader/workbook.py:88: UserWarning: File contains an invalid specificati

In [8]:
pd.DataFrame(usability).to_json('secondary/dataframe_usability.json')

In [33]:
from copy import copy 

usability = pd.read_json('secondary/dataframe_usability.json')
print(usability.shape)
usability = usability[usability.rows > 0] #remove those that are unusable
usability = usability[usability['columns'] > 0]
print(usability.shape)

(2178, 4)
(2116, 4)


In [30]:
usability

,rows,columns,path,sheet_name_counter
0,13,4,/economy/nationalaccounts/uksectoraccounts/dat...,0
1,29,4,/economy/nationalaccounts/uksectoraccounts/dat...,1
2,112,8,/economy/nationalaccounts/uksectoraccounts/dat...,2
3,49,11,/economy/inflationandpriceindices/datasets/con...,0
4,482,22,/businessindustryandtrade/internationaltrade/d...,0
...,...,...,...,...
2173,34,82,/economy/governmentpublicsectorandtaxes/public...,17
2174,46,17,/economy/governmentpublicsectorandtaxes/public...,18
2175,46,10,/economy/governmentpublicsectorandtaxes/public...,19
2176,54,22,/economy/governmentpublicsectorandtaxes/public...,20


In [31]:
og_usability

,rows,columns,path,sheet_name_counter
0,13,4,/economy/nationalaccounts/uksectoraccounts/dat...,0
1,29,4,/economy/nationalaccounts/uksectoraccounts/dat...,1
2,112,8,/economy/nationalaccounts/uksectoraccounts/dat...,2
3,49,11,/economy/inflationandpriceindices/datasets/con...,0
4,482,22,/businessindustryandtrade/internationaltrade/d...,0
...,...,...,...,...
2173,34,82,/economy/governmentpublicsectorandtaxes/public...,17
2174,46,17,/economy/governmentpublicsectorandtaxes/public...,18
2175,46,10,/economy/governmentpublicsectorandtaxes/public...,19
2176,54,22,/economy/governmentpublicsectorandtaxes/public...,20
